3. Create a tutorial on an big data algorthim that was not already submitted as part of an another assignment or extra credit.

# Decision Tree Classifier

Decision Tree algorithm belongs to the family of supervised learning algorithms. Unlike other supervised learning algorithms, decision tree algorithm can be used for solving regression and classification problems too. In this tutorial, I will be showing you a PYTHON ONLY implementation for a Decision Tree Classifier.

Before we start on the implementation, let's discuss in brief about the Decision Tree Algorithm.

Decision Tree algorithm belongs to the family of supervised learning algorithms. The general motive of using Decision Tree is to create a training model which can use to predict class or value of target variables by learning decision rules inferred from prior data(training data).

#### Decision Tree Algorithm Pseudocode
1. Place the best attribute of the dataset at the root of the tree.
2. Split the training set into subsets. Subsets should be made in such a way that each subset contains data with the same value for an attribute.
3. Repeat step 1 and step 2 on each subset until you find leaf nodes in all the branches of the tree.

The Decision Tree Classifier uses the model built using the Decision Tree to classify records. It poses a series of carefully crafted questions about the attributes of the test record. Each time it receive an answer, a follow-up question is asked until a conclusion about the calss label of the record is reached.

### Implementation
I am following this tutorial: https://www.youtube.com/watch?v=LDRbO9a6XPU for this implementation.

This implementation uses a TOY DATASET. The data consists of columns that represent features of a fruit like color, diameter, taste and it also has a column for the label - which basically tells us the fruit's name.

In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [101]:
# Format: each row is an example.
# The last column is the label - fruit's name.
# First 3 Columns are the features of the fruits - color, diameter, taste, label
training_data = [
    ['Green', 3,'Sour', 'Apple'],
    ['Red', 3, 'Sweet','Apple'],
    ['Green', 1, 'Sweet','Grape'],
    ['Black', 1, 'Sweet','Grape'],
    ['Yellow', 2, 'Sour','Lemon'],
    ['Yellow', 1, 'Sour','Lemon'],
]

In [102]:
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "taste","label"]

In [103]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [104]:
unique_vals(training_data, 0) #Testing the function

{'Black', 'Green', 'Red', 'Yellow'}

The following functions will be use to count the number of records for each label/class and to check whether a feature value is numeric or not.

In [105]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [106]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 2}

In [107]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

print (is_numeric(7))
print (is_numeric('Red'))

True
False


### Building the Decision Tree

To build the decision tree, we'll be using a Decision Tree Learning Algorithm called CART (Classification and Regression Tree). There are other algorithms as well like C4.5, ID3, C5.0 and so on. These algorithms basically give procedures to decide which questions to ask and when.

The representation for the CART model is a binary tree. Each root node represents a single input variable and a split point on that variable which basically asks a True/False question depending on which it moves ahead in the tree with the aim of getting the purest groups with respect to the labels. We continue to divide the data using these True/False questions until there are no more questions to ask.

Given a dataset with two inputs (x) of height in centimeters and weight in kilograms the output of sex as male or female, below is a crude example of a binary decision tree (completely fictitious for demonstration purposes only).

<img src="Example-Decision-Tree.png">


https://machinelearningmastery.com/classification-and-regression-trees-for-machine-learning/



The Question class will be used to partition a dataset. This class just records a 'column number' (e.g., 0 for Color) and a
'column value' (e.g., Green). The 'match' method is used to compare the feature value in an example to the feature value stored in the question.

In [108]:
class Question:

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [109]:
#Checking for the question to be asked for value 3 in column 1 
print (Question(1, 3))
#Checking for the question to be asked for value 'Green' in column 0
print (Question(0, 'Green'))

Is diameter >= 3?
Is color == Green?


Let us now pick an example from the training set and see if it matches the question or not


In [110]:
example = training_data[0] #Taking the first record in the training data
q.match(example)

True

The partition function will be used to partition the dataset. For each row in the dataset, check if it matches the question. If so, add it to 'true rows', otherwise, add it to 'false rows'.


In [111]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

Let's partition the training data based on whether rows are Green. The true_rows should contain rows with value 'Green' in it. And false_rows should contain rest of the rows.

In [112]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
print(true_rows)
print()
print(false_rows)

[['Green', 3, 'Sour', 'Apple'], ['Green', 1, 'Sweet', 'Grape']]

[['Red', 3, 'Sweet', 'Apple'], ['Black', 1, 'Sweet', 'Grape'], ['Yellow', 2, 'Sour', 'Lemon'], ['Yellow', 1, 'Sour', 'Lemon']]


Now let us calculate the Gini Impurity for a list of rows. Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. 
Check https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity for more on Gini Impurity.

In [113]:
def gini(rows):
  
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [114]:
mixed = [['Apple'],['Grape']]
non_mixed = [['Lemon'],['Lemon']]
print (gini(mixed))
print (gini(non_mixed))

0.5
0.0


The gini Impurity when equal to 0.5 tells us that there's a 5% chance of misclassifying a random example we draw from the dataset.

Let's try gini impurity for a dataset with many different labels.

In [115]:
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
gini(lots_of_mixing)

0.7999999999999998

Information gain is used to decide which feature to split on at each step in building the tree. The following function gives  the uncertainty of the starting node, minus the weighted impurity of two child nodes.

The split with the highest information gain will be taken as the first split and the process will continue until all children nodes are pure, or until the information gain is 0.

In [116]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [117]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6666666666666665

In [118]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.08333333333333315

In [119]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1333333333333332

Let's now find the best question to ask by iterating over every feature / value and calculating the information gain.

In [120]:
def find_best_split(rows):

    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [121]:
best_gain, best_question = find_best_split(training_data)
best_question

Is diameter >= 3?

Let's create a Leaf node which classifies data. This holds a dictionary of class (e.g., "Apple") -> number of times it appears in the rows from the training data that reach this leaf.

In [122]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

Now let's create A Decision Node asks a question. This holds a reference to the question, and to the two child nodes.

In [123]:
class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

Now finally we build the tree using recursion.

The Rules of recursion are: 
1) Believe that it works. 
2) Start by checking for the base case (no further information gain). 
3) Prepare for giant stack traces.


In [124]:
def build_tree(rows):
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [125]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [126]:
my_tree = build_tree(training_data)

In [127]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Predict {'Apple': 2}
--> False:
  Is taste == Sour?
  --> True:
    Predict {'Lemon': 2}
  --> False:
    Predict {'Grape': 2}


Using the decision tree we just built, now we classify data.

In [128]:
def classify(row, node):

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [129]:
classify(training_data[0], my_tree)

{'Apple': 2}

In [130]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [131]:
print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [132]:
print_leaf(classify(training_data[4], my_tree))

{'Lemon': '100%'}

In [145]:
# Evaluate
testing_data = [
    ['Green', 2, 'Sour','Apple'],
    ['Yellow', 4, 'Sweet','Apple'],
    ['Red', 4, 'Sour','Grape'],
    ['Red', 1, 'Sweet','Grape'],
    ['Yellow', 3,'Sweet', 'Lemon'],
]

In [143]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Grape. Predicted: {'Apple': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '100%'}


As we can see that 2 predictions made are incorrect. This is mainly because of our training data was really small. 